In [18]:
import os
import sys

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm
import torch
import torch.nn as nn
from torch.nn import functional as F
import re

%matplotlib inline 
%load_ext autoreload
%autoreload 2

Length of knowledge: 100
The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [24]:
checkpt_dir = "../runs/image_exp_results/slurm_logs/"

from os import listdir
from os.path import isfile, join

runs = [f for f in listdir(checkpt_dir) if "1292809" in f]
runs = sorted(runs, key=lambda x: int(x.split("_")[1].replace(".out", "")))
runs[: 2]

['slurm-1292809_1.out', 'slurm-1292809_2.out']

In [45]:
def parse_log_file(lines):
    results = {
        "val_acc": [],
        "val_SC_acc": [],
        "val_constraint_acc": [],
        "test_acc": 0,
        "test_SC_acc": 0,
        "test_constraint_acc": 0,
    }
    for line in lines:
        if "Test" in line:
            _, acc, sc_acc, constr_acc = line.split("\t")
            acc = float(acc.replace("Acc ", ""))
            sc_acc = float(sc_acc.replace("AccSC ", ""))
            constr_acc = float(constr_acc.replace("ConstraintAcc ", "").strip())

            results["val_acc"].append(acc)
            results["val_SC_acc"].append(sc_acc)
            results["val_constraint_acc"].append(constr_acc)
            
    results["test_acc"] = results["val_acc"][-1]
    results["test_SC_acc"] = results["val_SC_acc"][-1]
    results["test_constraint_acc"] = results["val_constraint_acc"][-1]
    results["val_acc"] = results["val_acc"][:-4]
    results["val_SC_acc"] = results["val_SC_acc"][:-4]
    results["val_constraint_acc"] = results["val_constraint_acc"][:-4]
            
    return results

results = []
for file in runs:
    with open(checkpt_dir + file, "r") as f:
        results.append(parse_log_file(f.readlines()))

In [47]:
def print_results(results, name):
    acc = np.mean([r["test_acc"] for r in results])
    acc_sd = np.std([r["test_acc"] for r in results])
    
    SCacc = np.mean([r["test_SC_acc"] for r in results])
    SCacc_sd = np.std([r["test_SC_acc"] for r in results])
    
    const_acc = np.mean([r["test_constraint_acc"] for r in results])
    const_acc_sd = np.std([r["test_constraint_acc"] for r in results])
    
    print(f"Results for {name}")
    print(f"Acc: {acc} +- {acc_sd}")
    print(f"SC Acc: {SCacc} +- {SCacc_sd}")
    print(f"Const: {const_acc} +- {const_acc_sd}")
    print(" --------------------------------------- ")

In [48]:
full_baseline = results[:5]
print_results(full_baseline, name="full baseline")

Results for full baseline
Acc: 68.228 +- 0.39168354573558534
SC Acc: 78.61599999999999 +- 0.5005836593417762
Const: 77.336 +- 0.6591084887937617
 --------------------------------------- 


In [49]:
full_baseline = results[5:10]
print_results(full_baseline, name="full baseline SC only")

Results for full baseline SC only
Acc: 78.444 +- 0.25554647326856467
SC Acc: nan +- nan
Const: nan +- nan
 --------------------------------------- 


In [50]:
full_baseline = results[10:15]
print_results(full_baseline, name="hierarchical")

Results for hierarchical
Acc: 63.842 +- 0.3793889824441371
SC Acc: 80.098 +- 0.23583044756774107
Const: nan +- nan
 --------------------------------------- 


In [51]:
full_baseline = results[15:20]
print_results(full_baseline, name="DL2")

Results for DL2
Acc: 68.534 +- 0.3344009569364277
SC Acc: 79.05600000000001 +- 0.18227451824103022
Const: 80.762 +- 0.45305187340966036
 --------------------------------------- 


In [52]:
full_baseline = results[20:25]
print_results(full_baseline, name="multiplexnet")

Results for multiplexnet
Acc: 67.136 +- 0.6372315120896036
SC Acc: 80.552 +- 0.581357033156046
Const: 100.0 +- 0.0
 --------------------------------------- 
